# Analyse des performances des valeurs cotées à la BRVM

Ce notebook analyse les performances historiques des différentes valeurs cotées à la Bourse Régionale des Valeurs Mobilières (BRVM) ainsi que de l'indice BRVM Composite.

In [ ]:
# Importation des bibliothèques nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime, timedelta
import glob

# Configuration pour les visualisations
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 12
sns.set_palette("viridis")

# Désactiver les avertissements
import warnings
warnings.filterwarnings('ignore')

## 1. Chargement des données

Nous commençons par charger toutes les données historiques qui ont été scrappées auparavant.

In [ ]:
# Fonction pour charger toutes les données des fichiers CSV
def load_all_data(data_dir='../data'):
    """Charge toutes les données des fichiers CSV dans le dossier spécifié."""
    all_files = glob.glob(os.path.join(data_dir, '*.csv'))
    
    data_frames = {}
    
    for file_path in all_files:
        file_name = os.path.basename(file_path)
        symbol = file_name.split('_')[0]
        
        try:
            df = pd.read_csv(file_path)
            
            # Convertir la date en format datetime
            df['Date'] = pd.to_datetime(df['Date'])
            
            # S'assurer que les colonnes numériques le sont bien
            for col in ['Ouverture', 'Plus_Haut', 'Plus_Bas', 'Cloture', 'Volume']:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col], errors='coerce')
            
            # Trier par date
            df = df.sort_values('Date')
            
            data_frames[symbol] = df
            print(f"Chargé {len(df)} lignes pour {symbol}")
        except Exception as e:
            print(f"Erreur lors du chargement de {file_path}: {str(e)}")
    
    return data_frames

# Charger toutes les données
all_data = load_all_data()

In [ ]:
# Vérification des données chargées
print(f"Nombre total de valeurs chargées: {len(all_data)}")

# Afficher la liste des valeurs disponibles
sorted_symbols = sorted(all_data.keys())
print("Valeurs disponibles:")
for symbol in sorted_symbols:
    df = all_data[symbol]
    start_date = df['Date'].min().strftime('%d/%m/%Y')
    end_date = df['Date'].max().strftime('%d/%m/%Y')
    nb_records = len(df)
    print(f"  - {symbol}: {nb_records} enregistrements du {start_date} au {end_date}")

## 2. Analyse de la performance globale

Nous allons calculer les performances globales pour chaque valeur depuis le début de sa cotation.

In [ ]:
def calculate_performance(df):
    """Calcule les indicateurs de performance pour une valeur."""
    if df.empty or len(df) < 2:
        return {}
    
    # Calculer les rendements journaliers
    df['Rendement'] = df['Cloture'].pct_change()
    
    # Dates de début et de fin
    start_date = df['Date'].min()
    end_date = df['Date'].max()
    duration_days = (end_date - start_date).days
    duration_years = duration_days / 365.25
    
    # Prix initial et final
    initial_price = df.iloc[0]['Cloture']
    final_price = df.iloc[-1]['Cloture']
    
    # Performance globale
    total_return = (final_price / initial_price - 1) * 100
    
    # Performance annualisée
    annual_return = (((final_price / initial_price) ** (1 / duration_years)) - 1) * 100 if duration_years > 0 else 0
    
    # Volatilité (écart-type des rendements journaliers annualisé)
    volatility = df['Rendement'].std() * np.sqrt(252) * 100  # Annualisé en %
    
    # Ratio de Sharpe (en supposant un taux sans risque de 3%)
    risk_free_rate = 0.03
    sharpe_ratio = (annual_return / 100 - risk_free_rate) / (volatility / 100) if volatility > 0 else 0
    
    # Rendement max et min
    max_daily_return = df['Rendement'].max() * 100
    min_daily_return = df['Rendement'].min() * 100
    
    # Drawdown maximum
    df['Cumul'] = (1 + df['Rendement']).cumprod()
    df['Drawdown'] = df['Cumul'] / df['Cumul'].cummax() - 1
    max_drawdown = df['Drawdown'].min() * 100
    
    return {
        'start_date': start_date,
        'end_date': end_date,
        'duration_days': duration_days,
        'duration_years': duration_years,
        'initial_price': initial_price,
        'final_price': final_price,
        'total_return': total_return,
        'annual_return': annual_return,
        'volatility': volatility,
        'sharpe_ratio': sharpe_ratio,
        'max_daily_return': max_daily_return,
        'min_daily_return': min_daily_return,
        'max_drawdown': max_drawdown
    }

# Calculer les performances pour toutes les valeurs
performances = {}
for symbol, df in all_data.items():
    try:
        perf = calculate_performance(df)
        if perf:  # Si le calcul a réussi
            performances[symbol] = perf
    except Exception as e:
        print(f"Erreur lors du calcul des performances pour {symbol}: {str(e)}")

# Créer un DataFrame avec toutes les performances
perf_df = pd.DataFrame.from_dict(performances, orient='index')
perf_df.index.name = 'Symbole'

# Trier par performance totale
perf_df = perf_df.sort_values('total_return', ascending=False)

# Afficher les résultats
perf_df[['total_return', 'annual_return', 'volatility', 'sharpe_ratio', 'max_drawdown']].round(2)

## 3. Visualisation des performances

In [ ]:
# Graphique des performances totales
plt.figure(figsize=(16, 10))
ax = sns.barplot(x=perf_df.index, y='total_return', data=perf_df)
plt.title('Performance totale depuis le début de cotation (%)', fontsize=16)
plt.xticks(rotation=90)
plt.ylabel('Performance totale (%)')
plt.xlabel('Valeur')

# Ajouter les valeurs sur les barres
for i, v in enumerate(perf_df['total_return']):
    ax.text(i, v + (5 if v >= 0 else -20), f"{v:.1f}%", ha='center', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# Graphique des performances annualisées
plt.figure(figsize=(16, 10))
ax = sns.barplot(x=perf_df.index, y='annual_return', data=perf_df)
plt.title('Performance annualisée (%)', fontsize=16)
plt.xticks(rotation=90)
plt.ylabel('Performance annualisée (%)')
plt.xlabel('Valeur')

# Ajouter les valeurs sur les barres
for i, v in enumerate(perf_df['annual_return']):
    ax.text(i, v + (2 if v >= 0 else -5), f"{v:.1f}%", ha='center', fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# Risque vs Rendement
plt.figure(figsize=(14, 10))
ax = sns.scatterplot(x='volatility', y='annual_return', size='total_return', 
                      hue='sharpe_ratio', data=perf_df, sizes=(50, 500))

plt.title('Risque vs Rendement des valeurs de la BRVM', fontsize=16)
plt.xlabel('Volatilité annualisée (%)')
plt.ylabel('Rendement annualisé (%)')
plt.axhline(y=0, color='red', linestyle='--', alpha=0.5)
plt.axvline(x=0, color='red', linestyle='--', alpha=0.5)

# Ajouter des annotations pour chaque point
for symbol in perf_df.index:
    x = perf_df.loc[symbol, 'volatility']
    y = perf_df.loc[symbol, 'annual_return']
    plt.annotate(symbol, (x, y), fontsize=9, ha='center')

plt.colorbar(ax.collections[0], label="Ratio de Sharpe")
plt.tight_layout()
plt.show()

## 4. Évolution des indices BRVM

In [ ]:
# Visualiser l'évolution de l'indice BRVM-Composite
if 'BRVM-Composite' in all_data:
    brvm_composite = all_data['BRVM-Composite']
    
    plt.figure(figsize=(16, 8))
    plt.plot(brvm_composite['Date'], brvm_composite['Cloture'])
    plt.title('Évolution de l\'indice BRVM-Composite', fontsize=16)
    plt.xlabel('Date')
    plt.ylabel('Valeur de l\'indice')
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    
    # Calculer les rendements annuels
    brvm_composite['Year'] = brvm_composite['Date'].dt.year
    annual_returns = []
    
    for year, group in brvm_composite.groupby('Year'):
        start_price = group.iloc[0]['Cloture']
        end_price = group.iloc[-1]['Cloture']
        annual_return = (end_price / start_price - 1) * 100
        annual_returns.append({'Year': year, 'Return': annual_return})
    
    annual_returns_df = pd.DataFrame(annual_returns)
    
    # Graphique des rendements annuels
    plt.figure(figsize=(14, 8))
    ax = sns.barplot(x='Year', y='Return', data=annual_returns_df)
    plt.title('Rendements annuels de l\'indice BRVM-Composite (%)', fontsize=16)
    plt.ylabel('Rendement (%)')
    plt.xlabel('Année')
    
    # Ajouter les valeurs sur les barres
    for i, v in enumerate(annual_returns_df['Return']):
        ax.text(i, v + (1 if v >= 0 else -3), f"{v:.1f}%", ha='center')
    
    plt.tight_layout()
    plt.show()
else:
    print("Les données de l'indice BRVM-Composite ne sont pas disponibles.")

## 5. Comparaison des performances par secteur

In [ ]:
# Classification manuelle des secteurs (à adapter selon les données disponibles)
sectors = {
    'Banque': ['SGBCI', 'BOA', 'ECOBANK', 'SIB', 'NSIA', 'BICI', 'BDM', 'CORIS'],
    'Agro-industrie': ['SOGB', 'SAPH', 'PALC', 'SIFCA', 'SICOR', 'SUCRIVOIRE'],
    'Distribution': ['CFAO', 'BERNABE', 'VIVO', 'TOTAL'],
    'Services publics': ['SODECI', 'CIE', 'SONATEL', 'ONATEL'],
    'Industrie': ['NESTLE', 'SOLIBRA', 'SMB', 'UNIWAX', 'FILTISAC', 'AIR'],
    'Transport': ['BOLLORE', 'MOVIS', 'SETAO'],
    'Autres': []  # Pour les valeurs non classées
}

# Créer une table de correspondance symbole -> secteur
symbol_to_sector = {}
for sector, symbols in sectors.items():
    for symbol in symbols:
        symbol_to_sector[symbol] = sector

# Ajouter les valeurs non classées au secteur 'Autres'
for symbol in all_data.keys():
    if symbol not in symbol_to_sector and not symbol.startswith('BRVM'):
        symbol_to_sector[symbol] = 'Autres'

# Ajouter le secteur au DataFrame des performances
perf_df['Secteur'] = perf_df.index.map(lambda x: symbol_to_sector.get(x, 'Indice' if x.startswith('BRVM') else 'Autres'))

# Analyser les performances par secteur
sector_perf = perf_df.groupby('Secteur').agg({
    'total_return': 'mean',
    'annual_return': 'mean',
    'volatility': 'mean',
    'sharpe_ratio': 'mean',
    'max_drawdown': 'mean'
}).round(2)

# Trier par performance annualisée
sector_perf = sector_perf.sort_values('annual_return', ascending=False)

# Afficher les résultats
sector_perf

In [ ]:
# Graphique des performances moyennes par secteur
plt.figure(figsize=(14, 8))
ax = sns.barplot(x=sector_perf.index, y='annual_return', data=sector_perf)
plt.title('Performance annualisée moyenne par secteur (%)', fontsize=16)
plt.xticks(rotation=45)
plt.ylabel('Performance annualisée moyenne (%)')
plt.xlabel('Secteur')

# Ajouter les valeurs sur les barres
for i, v in enumerate(sector_perf['annual_return']):
    ax.text(i, v + (1 if v >= 0 else -3), f"{v:.1f}%", ha='center')

plt.tight_layout()
plt.show()

## 6. Analyse des corrélations entre les valeurs

In [ ]:
# Créer un DataFrame avec les prix de clôture de toutes les valeurs
# sur une période commune
def create_price_matrix(all_data, min_periods=252):  # Au moins 1 an de données
    """Crée une matrice de prix pour toutes les valeurs sur une période commune."""
    # Extraire les séries de prix de clôture
    price_series = {}
    for symbol, df in all_data.items():
        if len(df) >= min_periods and not symbol.startswith('BRVM'):  # Exclure les indices
            price_series[symbol] = pd.Series(df['Cloture'].values, index=df['Date'])
    
    # Créer un DataFrame avec toutes les séries
    price_df = pd.DataFrame(price_series)
    
    # Garder uniquement les valeurs avec suffisamment de données
    valid_columns = price_df.columns[price_df.count() >= min_periods]
    price_df = price_df[valid_columns]
    
    return price_df

# Créer la matrice des prix
price_matrix = create_price_matrix(all_data)

if not price_matrix.empty and price_matrix.shape[1] > 1:
    # Calculer les rendements quotidiens
    returns_matrix = price_matrix.pct_change().dropna()
    
    # Calculer la matrice de corrélation
    corr_matrix = returns_matrix.corr()
    
    # Afficher la heatmap des corrélations
    plt.figure(figsize=(16, 14))
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    sns.heatmap(corr_matrix, mask=mask, annot=False, cmap='coolwarm', center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5})
    plt.title('Matrice de corrélation des rendements quotidiens', fontsize=16)
    plt.tight_layout()
    plt.show()
    
    # Trouver les paires les plus corrélées et les moins corrélées
    # Exclure les auto-corrélations (diagonale)
    corr_no_diag = corr_matrix.where(~np.eye(len(corr_matrix), dtype=bool))
    
    # Paires les plus corrélées positivement
    most_corr = corr_no_diag.unstack().sort_values(ascending=False).head(10)
    print("Paires de valeurs les plus corrélées positivement:")
    for (stock1, stock2), corr in most_corr.items():
        print(f"{stock1} - {stock2}: {corr:.3f}")
    
    # Paires les plus corrélées négativement
    least_corr = corr_no_diag.unstack().sort_values().head(10)
    print("\nPaires de valeurs les plus corrélées négativement:")
    for (stock1, stock2), corr in least_corr.items():
        print(f"{stock1} - {stock2}: {corr:.3f}")
    
else:
    print("Pas assez de données pour calculer les corrélations.")

## 7. Synthèse et conclusion

### Synthèse des meilleures performances

Affichons les 10 valeurs avec les meilleures performances annualisées:

In [ ]:
# Top 10 des meilleures performances annualisées
top_10 = perf_df.sort_values('annual_return', ascending=False).head(10)
top_10[['Secteur', 'total_return', 'annual_return', 'volatility', 'sharpe_ratio', 'duration_years']].round(2)

### Synthèse des valeurs avec le meilleur ratio risque/rendement

Affichons les 10 valeurs avec les meilleurs ratios de Sharpe (rendement ajusté au risque):

In [ ]:
# Top 10 des meilleurs ratios de Sharpe
top_10_sharpe = perf_df.sort_values('sharpe_ratio', ascending=False).head(10)
top_10_sharpe[['Secteur', 'annual_return', 'volatility', 'sharpe_ratio', 'max_drawdown', 'duration_years']].round(2)

### Conclusion

En conclusion, cette analyse des performances historiques des valeurs cotées à la BRVM nous a permis de dégager plusieurs observations importantes:

1. **Performance globale du marché**: L'indice BRVM-Composite montre une tendance globale de [à compléter selon les résultats].

2. **Secteurs performants**: Les secteurs qui ont offert les meilleures performances annualisées sont [à compléter selon les résultats].

3. **Valeurs individuelles**: Les valeurs ayant démontré les meilleures performances sont [à compléter selon les résultats].

4. **Rapport risque/rendement**: En termes de rapport risque/rendement (ratio de Sharpe), les valeurs les plus intéressantes sont [à compléter selon les résultats].

5. **Corrélations**: Nous avons identifié des corrélations fortes entre certaines valeurs, notamment [à compléter selon les résultats], ce qui offre des perspectives intéressantes en termes de diversification.

Pour les investisseurs intéressés par le marché de la BRVM, ces résultats suggèrent qu'il pourrait être avantageux de [à compléter selon les recommandations basées sur les analyses].

Cette analyse devra être complétée par une mise à jour régulière des données et une prise en compte des facteurs macroéconomiques affectant les marchés financiers d'Afrique de l'Ouest.